<a href="https://colab.research.google.com/drive/1tk-hGUVytDUfMSBuw8V8K6UnWZ_yY1ab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp drive/'My Drive'/wonderland.txt .

In [4]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

Using TensorFlow backend.


In [5]:
raw_text[:1000]

"{\\rtf1\\ansi\\ansicpg1252\\cocoartf1348\\cocoasubrtf170\n{\\fonttbl\\f0\\fmodern\\fcharset0 courier;}\n{\\colortbl;\\red255\\green255\\blue255;\\red0\\green0\\blue0;}\n\\margl1440\\margr1440\\vieww10800\\viewh8400\\viewkind0\n\\deftab720\n\\pard\\pardeftab720\n\n\\f0\\fs24 \\cf2 \\expnd0\\expndtw0\\kerning0\n\\outl0\\strokewidth0 \\strokec2 project gutenberg's alice's adventures in wonderland, by lewis carroll\\\n\\\nthis ebook is for the use of anyone anywhere at no cost and with\\\nalmost no restrictions whatsoever.  you may copy it, give it away or\\\nre-use it under the terms of the project gutenberg license included\\\nwith this ebook or online at www.gutenberg.org\\\n\\\n\\\ntitle: alice's adventures in wonderland\\\n\\\nauthor: lewis carroll\\\n\\\nposting date: june 25, 2008 [ebook #11]\\\nrelease date: march, 1994\\\n[last updated: december 20, 2011]\\\n\\\nlanguage: english\\\n\\\n\\\n*** start of this project gutenberg ebook alice's adventures in wonderland ***\\\n\\\n\\\n

Removed Punctuation from source text for better results

In [0]:
import re
raw_text = re.sub(r'[^\w\s\t\n]','',raw_text)

In [13]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
chars

['\n',
 ' ',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [14]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  155125
Total Vocab:  39


**Article is split by sentences and each set of 100 characters in a sentence is used for training. <br>
If the length of sentence is less than 100 it is padded using keras pad_sequences**

In [15]:
sentence = raw_text.split("\n\n")
from keras.preprocessing.sequence import pad_sequences
sent = ([x for x in sentence if sum(c.isalpha() for c in x)>1])
dataX = []
dataY = []
seq_length = 101
for s in sent:
  if len(s)<101:
    x = [char_to_int[a] for a in s]
    x = pad_sequences([x], maxlen=seq_length)
    dataX.append([a for a in x[0][:-1]])
    dataY.append(x[0][-1])
  else:
    for i in range(0, len(s) - seq_length, 1):
      x = [char_to_int[a] for a in s[i:i + seq_length]] 
      dataX.append([a for a in x[:-1]])
      dataY.append(x[-1])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
print(len(dataX))
X = numpy.reshape(numpy.array(dataX), (n_patterns, seq_length-1, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)      

Total Patterns:  82392
82392


In [0]:
# define the LSTM model
model = Sequential()
model.add(Dropout(0.1))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [17]:
# fit the model
model.fit(X, y, epochs=100, batch_size=256, callbacks=callbacks_list)

Epoch 1/100
82392/82392 [==============================] - 90s 1ms/step - loss: 2.8950

Epoch 00001: loss improved from inf to 2.89498, saving model to weights-improvement-01-2.8950-bigger.hdf5
Epoch 2/100
82392/82392 [==============================] - 89s 1ms/step - loss: 2.7933

Epoch 00002: loss improved from 2.89498 to 2.79331, saving model to weights-improvement-02-2.7933-bigger.hdf5
Epoch 3/100
82392/82392 [==============================] - 89s 1ms/step - loss: 2.7132

Epoch 00003: loss improved from 2.79331 to 2.71321, saving model to weights-improvement-03-2.7132-bigger.hdf5
Epoch 4/100
82392/82392 [==============================] - 89s 1ms/step - loss: 2.6598

Epoch 00004: loss improved from 2.71321 to 2.65978, saving model to weights-improvement-04-2.6598-bigger.hdf5
Epoch 5/100
82392/82392 [==============================] - 89s 1ms/step - loss: 2.6083

Epoch 00005: loss improved from 2.65978 to 2.60831, saving model to weights-improvement-05-2.6083-bigger.hdf5
Epoch 6/100
82

In [18]:
import sys
int_to_char = dict((i, c) for i, c in enumerate(chars))
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" hing but outoftheway
things to happen that it seemed quite dull and stupid for life to go on
in the  "
inanlny anlces neepinert tuojthing a pots anl tayaii ani eoy wet snlce ayayioy anl toecsned foisaclliny on tuesene  t teragling at  a lottt ou toeasnenei anlce fosi thpplencerine whamardenineer asd asdhes l c motwelint mnlet  oatueled sr teraglen fooct  nicpinerr cnlcelert c dopsrttott wo yhisellny iol tros all a0 eeysa  f folct neesinelere tr tepark difuritatty ofppus tr teaty iolctinnsid wi ys ouott  t teragrapcerirs perageepine  whems tep anl e pote oncllned iitecti  toated anl eopweu tndiisg
Done.
